# ICNet 不变性约束发现偏微分方程

## 环境安装

本案例要求 MindSpore >= 2.0.0 版本以调用如下接口: mindspore.jit, mindspore.jit_class, mindspore.data_sink。具体请查看MindSpore安装。

此外，你需要安装 MindFlow >=0.1.0 版本。如果当前环境还没有安装，请按照下列方式选择后端和版本进行安装。

In [4]:
mindflow_version = "0.1.0"  # update if needed
# GPU Comment out the following code if you are using NPU.
!pip uninstall -y mindflow-gpu
!pip install mindflow-gpu==$mindflow_version

# NPU Uncomment if needed.
# !pip uninstall -y mindflow-ascend
# !pip install mindflow-ascend==$mindflow_version

## 背景介绍

由偏微分方程描述的物理规律广泛存在于自然环境之中，物理系统的计算与模拟依赖于准确的基本方程和模型，传统方法推导控制方程主要基于第一性原理，例如Navier-Stokes方程基于动量守恒，传统方法难点在于复杂动力学的模型与方程常常难以推导，例如多相流、神经科学以及生物科学等，在大数据时代，通过人工智能的方法从数据中挖掘控制方程成为一种新的研究思路。已有的数据驱动发现方程的方法依然存在一定的局限性，目前构建过完备库的候选项时缺乏指导原则，无法保证发现的方程满足基本的物理要求，同时在处理复杂多维系统时候选库过大，而难以发现出简约准确的方程。考虑到基本的物理要求（不变性，守恒性等）是很多物理问题出发的基石，因此有必要研究如何在发现方程中施加物理约束。

## 模型框架

模型框架图如下所示:

![ICNet](images/ICNet.png)

图中：
A. 嵌入不变性约束至发现偏微分方程框架中的推导过程示意图；
B. 不变性约束发现偏微分方程的神经网络模块，利用神经网络自动微分求出构建不变性候选函数所需要的偏导数，损失函数包括数据损失Data loss，不变性损失Invariance loss以及增强稀疏性的正则化损失Regularization loss。

## 准备环节

实践前，确保已经正确安装合适版本的MindSpore。如果没有，可以通过：

* [MindSpore安装页面](https://www.mindspore.cn/install) 安装MindSpore。

## 数据集的准备

数据集下载地址：[ICNet/dataset](https://download-mindspore.osinfra.cn/mindscience/mindflow/dataset/applications/research/ICNet/)。将数据集保存在`./dataset`路径下。

## 模型训练

引入代码包

In [ ]:
import argparse
import time
import numpy as np

import mindspore as ms
from mindspore import set_seed, context, nn
from src.network import InvarianceConstrainedNN, InvarianceConstrainedNN_STRdige
from src.datasets import read_training_data, print_pde

模型相关参数的设置以及训练模型的定义

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--model_name', type=str, default='ICNet')
parser.add_argument('--case', type=str, default='Kuramoto-Sivashinsky equation')
parser.add_argument('--device', type=str, default='GPU')    #default='GPU' or 'Ascend'
parser.add_argument('--device_id', type=str, default=3)
parser.add_argument('--init_steps', type=str, default=0)
parser.add_argument('--stop_steps', type=str, default=150)
parser.add_argument('--time_steps', type=str, default=50)
parser.add_argument('--load_params', type=str, default='True')
parser.add_argument('--second_path', type=str, default='pretrain')
parser.add_argument('--data_name', type=str, default='KS.mat')
parser.add_argument('--description_ks', type=str, default=['uu_x', '1', 'u_x', 'u_xx', 'u_xxx', 'u_xxxx'])
parser.add_argument('--network_size', type=int, default=[2] + 8*[40] + [1])
parser.add_argument('--learning_rate', type=int, default=[0.001, 0.0005, 1.0e-04, 1.0e-05])
parser.add_argument('--epochs', type=int, default=[30e4, 30e4, 1e4, 1e4])
parser.add_argument('--BatchNo', type=int, default=1)
parser.add_argument('--lam', type=float, default=1e-5)
parser.add_argument('--d_tol', type=float, default=1.0)
args = parser.parse_known_args()[0]

model_name = args.model_name
case = args.case
device = args.device
device_id = args.device_id
network_size = args.network_size
learning_rate = args.learning_rate
epochs = args.epochs
BatchNo = args.BatchNo
load_params = args.load_params
second_path = args.second_path
description_ks = args.description_ks
lam = args.lam
d_tol = args.d_tol

use_ascend = context.get_context(attr_key='device_target') == "Ascend"

if use_ascend:
    msfloat_type = ms.float16
else:
    msfloat_type = ms.float32

context.set_context(mode=context.GRAPH_MODE, save_graphs=False, device_target=device, device_id=device_id)

X_u_train, u_train, X_f_train = read_training_data(args)

model_pretrain = InvarianceConstrainedNN(X_u_train, u_train, X_f_train, network_size, BatchNo, use_ascend, msfloat_type)

设置种子

In [3]:
np.random.seed(123456)
set_seed(123456)

代码训练与输出结果部分

In [ ]:
def train(model, niter, lr):
    # Get the gradients function

    params = model.dnn.trainable_params()
    params.append(model.lambda_u)
    params.append(model.lambda_uux)

    optimizer_Adam = nn.Adam(params, learning_rate=lr)

    grad_fn = ms.value_and_grad(model.loss_fn, None, optimizer_Adam.parameters, has_aux=True)

    model.dnn.set_train()

    start_time = time.time()

    for epoch in range(1, 1+niter):
        (loss, loss_u, loss_f_u, loss_lambda_u), grads = grad_fn(model.x, model.t, model.x_f, model.t_f, model.u)

        optimizer_Adam(grads)

        if epoch % 10 == 0:
            elapsed = time.time() - start_time
            print('It: %d, Loss: %.3e, loss_u:  %.3e, loss_f:  %.3e, loss_lambda:  %.3e, Lambda_uux: %.3f, Lambda_uxx: %.3f, Lambda_uxxxx: %.3f, Time: %.2f'  %\
                    (epoch, loss.item(), loss_u.item(), loss_f_u.item(), loss_lambda_u.item(),
                     model.lambda_uux.item(), model.lambda_u[2].item(), model.lambda_u[4].item(), elapsed))

            initial_size = 5

            loss_history_Adam_Pretrain = np.empty([0])
            loss_u_history_Adam_Pretrain = np.empty([0])
            loss_f_u_history_Adam_Pretrain = np.empty([0])
            loss_lambda_u_history_Adam_Pretrain = np.empty([0])

            lambda_u_history_Adam_Pretrain = np.zeros((initial_size, 1))
            lambda_uux_history_Adam_Pretrain = np.zeros((1, 1))

            loss_history_Adam_Pretrain = np.append(loss_history_Adam_Pretrain, loss.numpy())
            lambda_u_history_Adam_Pretrain = np.append(lambda_u_history_Adam_Pretrain, model.lambda_u.numpy(), axis=1)
            loss_u_history_Adam_Pretrain = np.append(loss_u_history_Adam_Pretrain, loss_u.numpy())
            loss_f_u_history_Adam_Pretrain = np.append(loss_f_u_history_Adam_Pretrain, loss_f_u.numpy())
            loss_lambda_u_history_Adam_Pretrain = np.append(loss_lambda_u_history_Adam_Pretrain, loss_lambda_u.numpy())

            lambda_uux_new = np.array([model.lambda_uux.numpy()])
            lambda_uux_history_Adam_Pretrain = np.append(lambda_uux_history_Adam_Pretrain, lambda_uux_new, axis=1)

            start_time = time.time()
    np.save(f'Loss-Coe/{second_path}/loss_history_Adam_Pretrain', loss_history_Adam_Pretrain)
    np.save(f'Loss-Coe/{second_path}/loss_u_history_Adam_Pretrain', loss_u_history_Adam_Pretrain)
    np.save(f'Loss-Coe/{second_path}/loss_f_u_history_Adam_Pretrain', loss_f_u_history_Adam_Pretrain)
    np.save(f'Loss-Coe/{second_path}/loss_lambda_u_history_Adam_Pretrain', loss_lambda_u_history_Adam_Pretrain)

    np.save(f'Loss-Coe/{second_path}/lambda_u_history_Adam_Pretrain', lambda_u_history_Adam_Pretrain)
    np.save(f'Loss-Coe/{second_path}/lambda_uux_history_Adam_Pretrain', lambda_uux_history_Adam_Pretrain)

运行训练与保存训练模型

In [ ]:
for epoch, lr in zip(epochs, learning_rate):
    train(model_pretrain, int(epoch), lr)
ms.save_checkpoint(model_pretrain.dnn, f'model/{second_path}/model.ckpt')

It: 0, Loss: 1.059e+00, loss_u:  1.059e+00, loss_f:  2.203e-07, loss_lambda:  0.000e+00, Lambda_uux: -0.000, Lambda_uxx: -0.000, Lambda_uxxxx: -0.000, Time: 9.18
It: 10, Loss: 1.045e+00, loss_u:  1.045e+00, loss_f:  4.615e-05, loss_lambda:  1.824e-09, Lambda_uux: 0.004, Lambda_uxx: 0.003, Lambda_uxxxx: -0.003, Time: 2.72
It: 20, Loss: 1.040e+00, loss_u:  1.039e+00, loss_f:  1.004e-04, loss_lambda:  3.732e-09, Lambda_uux: 0.011, Lambda_uxx: 0.005, Lambda_uxxxx: 0.001, Time: 2.72
It: 30, Loss: 1.034e+00, loss_u:  1.034e+00, loss_f:  2.701e-04, loss_lambda:  6.927e-09, Lambda_uux: 0.004, Lambda_uxx: -0.005, Lambda_uxxxx: 0.009, Time: 2.70
It: 40, Loss: 1.029e+00, loss_u:  1.028e+00, loss_f:  6.826e-04, loss_lambda:  1.216e-08, Lambda_uux: -0.009, Lambda_uxx: -0.019, Lambda_uxxxx: 0.023, Time: 2.73
It: 50, Loss: 1.022e+00, loss_u:  1.021e+00, loss_f:  1.178e-03, loss_lambda:  1.785e-08, Lambda_uux: -0.020, Lambda_uxx: -0.033, Lambda_uxxxx: 0.035, Time: 2.70
It: 60, Loss: 1.017e+00, loss_u:

保存最后一次训练的可学习参数用于方程发现

In [ ]:
lambda_uux_value = model_pretrain.lambda_uux.numpy()
lambda_u_value = model_pretrain.lambda_u.numpy()
np.save(f'Loss-Coe/{second_path}/lambda_uux_value', lambda_uux_value)
np.save(f'Loss-Coe/{second_path}/lambda_u_value', lambda_u_value)

训练结束后直接进行方程发现可能会超出显存，因此需要根据计算机显存大小判断是否需要重新加载模型进行方程发现

In [ ]:
if load_params:
    lambda_u_value = np.load(f'Loss-Coe/{second_path}/lambda_u_value.npy')
    lambda_uux_value = np.load(f'Loss-Coe/{second_path}/lambda_uux_value.npy')
    model_ICCO = InvarianceConstrainedNN_STRdige(X_u_train, u_train, X_f_train, network_size, BatchNo, lambda_u_value, lambda_uux_value, load_params, second_path, msfloat_type)
else:
    model_ICCO = InvarianceConstrainedNN_STRdige(X_u_train, u_train, X_f_train, network_size, BatchNo, lambda_u_value, lambda_uux_value, load_params, second_path, msfloat_type)

In [ ]:
lambda_u_STRidge = model_ICCO.call_trainstridge(lam, d_tol)

In [ ]:
# GPU results
print_pde(lambda_uux_value, lambda_u_STRidge, description_ks, ut='u_t')

u_t = (0.973947)uu_x
   (-0.967219)u_xx
    + (-0.967183)u_xxxx
   
